https://www.analyticsvidhya.com/blog/2024/06/finetuning-llama-3-for-sequence-classification/

In [1]:
!pip install -q transformers accelerate trl bitsandbytes datasets evaluate
!pip install -q peft scikit-learn
!pip install -U "huggingface_hub[cli]"

In [2]:
with open("../private_/hf_read_token", "r") as f:
  token = f.readline()

hf_read = token

with open("../private_/hf_write_token", "r") as f:
  token = f.readline()

hf_write = token

In [3]:
!huggingface-cli login --token $hf_read

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/arjunsohur/.cache/huggingface/token
Login successful


In [4]:
from convokit import Corpus, download
from datasets import Dataset, DatasetDict

import torch
from transformers import BitsAndBytesConfig, AutoModelForSequenceClassification

import numpy as np

In [5]:
from huggingface_hub import HfApi, login

# not sure if the r and w tokens are needed but oh well
login(token=hf_write)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/arjunsohur/.cache/huggingface/token
Login successful


In [6]:
%%capture
corpus = Corpus(filename=download("winning-args-corpus"))

In [7]:
import pandas as pd

ids = corpus.get_utterance_ids()
print("Len of ids", len(ids))

SPEAKER_BLACKLIST = ['DeltaBot','AutoModerator']
training_trios = []

for id in ids:
  ut = corpus.get_utterance(id)
  if ut.reply_to == ut.conversation_id and (ut.meta['success'] == 1 or ut.meta['success'] == 0) and (ut.speaker.id not in SPEAKER_BLACKLIST):
    op = corpus.get_utterance(ut.conversation_id).text
    x = ut.text
    y = ut.meta['success']

    training_trios += [(op, x, y)]

print(len(training_trios))

train_len = len(training_trios)

ones = 0
zeros = 0
total = 0

def formatting_prompts_func(training_trios):
    texts = []
    targets = []

    total = 0
    ones = 0
    zeros = 0

    for trio in training_trios:
        op, x, y = trio
        instruction = "Please determine if the following argument is successful based on the original post.  Output 1 for successful and 0 for unsuccessful.  Only output the one number, NOTHING ELSE."
        input_context = f"Original post: {op}\nArgument: {x}"

        texts.append(input_context)
        targets.append(y)

        if y:
           ones+=1
        else:
           zeros+=1
        total += 1

    return texts, targets, ones, zeros, total

# Format the data
texts, targets, ones, zeros, total = formatting_prompts_func(training_trios)

len_text = len(texts)
len_text = int(len_text*(15/8106))

v_start = int(len_text * 0.8)
v_end = int(len_text * 0.9)

train = {"text": texts[:v_start], "label": targets[:v_start]}
val = {"text": texts[v_start:v_end], "label":targets[v_start:v_end]}
test = {"text": texts[v_end:len_text], "label":targets[v_end:len_text]}

train_ds = Dataset.from_dict(train)
val_ds = Dataset.from_dict(val)
test_ds = Dataset.from_dict(test)

train_df = pd.DataFrame.from_dict(train_ds)
val_df = pd.DataFrame.from_dict(val_ds)
test_df = pd.DataFrame.from_dict(test_ds)

dataset = DatasetDict({
   'train': train_ds,
   'val': val_ds,
   'test': test_ds
})

print(dataset)

Len of ids 293297
8106
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 12
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 1
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2
    })
})


In [8]:
class_weights=(1/train_df.label.value_counts(normalize=True).sort_index()).tolist()
class_weights=torch.tensor(class_weights)
class_weights=class_weights/class_weights.sum()
class_weights

tensor([0.5833, 0.4167])

In [9]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, 
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_use_double_quant = True, 
    bnb_4bit_compute_dtype = torch.bfloat16 
)

model_name = "meta-llama/Meta-Llama-3-8B"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    num_labels=2,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

lora_config = LoraConfig(
    r = 16, 
    lora_alpha = 8,
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05, 
    bias = 'none',
    task_type = 'SEQ_CLS'
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [11]:
from transformers import AutoTokenizer

model_name = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.config.pretraining_tp = 1

In [13]:
sentences = test_df.text.tolist()

batch_size = 32  

all_outputs = []

for i in range(0, len(sentences), batch_size):
    batch_sentences = sentences[i:i + batch_size]

    inputs = tokenizer(batch_sentences, return_tensors="pt", 
    padding=True, truncation=True, max_length=512)

    inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu') for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        all_outputs.append(outputs['logits'])
        
final_outputs = torch.cat(all_outputs, dim=0)
test_df['predictions']=final_outputs.argmax(axis=1).cpu().numpy()

In [14]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import balanced_accuracy_score, classification_report

def get_metrics_result(test_df):
    y_test = test_df.label
    y_pred = test_df.predictions

    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    print("Balanced Accuracy Score:", balanced_accuracy_score(y_test, y_pred))
    print("Accuracy Score:", accuracy_score(y_test, y_pred))

get_metrics_result(test_df)

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       1.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0

Balanced Accuracy Score: 0.0
Accuracy Score: 0.0


In [15]:
def data_preprocesing(row):
    return tokenizer(row['text'], truncation=True, max_length=1000)

tokenized_data = dataset.map(data_preprocesing, batched=True, 
remove_columns=['text'])
tokenized_data.set_format("torch")

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [16]:
from transformers import DataCollatorWithPadding

collate_fn = DataCollatorWithPadding(tokenizer=tokenizer)

In [17]:
def compute_metrics(evaluations):
    predictions, labels = evaluations
    predictions = np.argmax(predictions, axis=1)
    return {'balanced_accuracy' : balanced_accuracy_score(predictions, labels),
    'accuracy':accuracy_score(predictions,labels)}

In [18]:
from transformers import Trainer, TrainingArguments
import torch.nn.functional as F
import logging


class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels").long()
        outputs = model(**inputs)
        logits = outputs.get('logits')

        if self.class_weights is not None:
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)
        else:
            loss = F.cross_entropy(logits, labels)

        print(f"LOSS: {loss}")

        return (loss, outputs) if return_outputs else loss

In [20]:
training_args = TrainingArguments(
    output_dir='persuasion_classification',
    learning_rate=1e-6,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    max_steps=2000,  # 500 steps * 4 "epochs" = 2000 total steps
    logging_steps=10,
    weight_decay=0.05,
    evaluation_strategy='steps',
    eval_steps=10,
    save_steps=500,
    load_best_model_at_end=True,
    report_to="none",
    lr_scheduler_type='linear',
    gradient_accumulation_steps=1,
    warmup_steps=500,
    dataloader_drop_last=False,  # Ensure all data is used
    remove_unused_columns=False  # Prevent potential issues with custom datasets
)

In [23]:
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_data['train'],
    eval_dataset = [tokenized_data['val'], tokenized_data['train']],
    tokenizer = tokenizer,
    data_collator = collate_fn,
    compute_metrics = compute_metrics,
    class_weights=class_weights,
)

TypeError: Trainer.__init__() got an unexpected keyword argument 'multiple_eval_dataset'

In [22]:
train_result = trainer.train()

/home/arjunsohur/miniconda3/envs/persuasion_env3/lib/python3.12/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


LOSS: 0.028975751250982285


Step,Training Loss,Validation Loss


LOSS: 2.4813897609710693
LOSS: 0.5152827501296997
LOSS: 1.2240965366363525
LOSS: 1.2300755977630615
LOSS: 0.1303340047597885
LOSS: 0.8618661761283875
LOSS: 2.3134243488311768
LOSS: 0.040579456835985184
LOSS: 0.907604992389679


AttributeError: 'list' object has no attribute 'keys'

In [ ]:
def generate_predictions(model,df_test):
    sentences = df_test.text.tolist()
    batch_size = 32  
    all_outputs = []

    for i in range(0, len(sentences), batch_size):

        batch_sentences = sentences[i:i + batch_size]

        inputs = tokenizer(batch_sentences, return_tensors="pt", 
        padding=True, truncation=True, max_length=512)

        inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu') 
        for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            all_outputs.append(outputs['logits'])
        
    final_outputs = torch.cat(all_outputs, dim=0)
    df_test['predictions']=final_outputs.argmax(axis=1).cpu().numpy()

generate_predictions(model,test_df)
get_metrics_result(test_df)

Classification Report:
              precision    recall  f1-score   support

           0       0.46      0.53      0.49        30
           1       0.50      0.42      0.46        33

    accuracy                           0.48        63
   macro avg       0.48      0.48      0.48        63
weighted avg       0.48      0.48      0.47        63

Balanced Accuracy Score: 0.47878787878787876
Accuracy Score: 0.47619047619047616


In [ ]:
generate_predictions(model,train_df)
get_metrics_result(train_df)

Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.50      0.51       237
           1       0.56      0.57      0.57       263

    accuracy                           0.54       500
   macro avg       0.54      0.54      0.54       500
weighted avg       0.54      0.54      0.54       500

Balanced Accuracy Score: 0.5360173910253325
Accuracy Score: 0.538


In [ ]:
test_df.head(20)

In [ ]:
# model.push_to_hub("ArjunSohur/argument_classification")